In [7]:
from bbq.analyse import *
import pandas as pd
from datetime import datetime, timedelta
from talib.abstract import *
import talib
from bbq.data.stockdb import StockDB
from bbq.data.funddb import FundDB
from pyecharts.charts import *
from pyecharts import options as opts
from pyecharts.commons.utils import JsCode
import json
import scipy.signal as signal
from sklearn.linear_model import LinearRegression
import numpy as np
from bbq.analyse.tools import *
from bbq.analyse.plot import *
import bbq.log as log
from bbq.selector.strategy import strategies

log.setup_logger(level='error')

db_stock = StockDB()
db_fund = FundDB()
db_stock.init()
db_fund.init()


data = await db_stock.load_stock_daily(filter={'code': 'sz002132'}, limit=60, sort=[('trade_date', -1)])
data = data[::-1]

name = await db_stock.load_stock_info(filter={'code': 'sz002132'}, projection=['name'])
data['name'] = name.iloc[0]['name']


data_fund = await db_fund.load_fund_daily(filter={'code': '159949'}, limit=60, sort=[('trade_date', -1)])
data_fund = data_fund[::-1]

name_fund = await db_fund.load_fund_info(filter={'code': '159949'}, projection=['short_name'])
data_fund['name'] = name_fund.iloc[0]['short_name']

print('doneq')


doneq


In [2]:

k_data = list(zip(data['open'], data['close'], data['low'], data['high']))
trade_date = [d.strftime('%Y-%m-%d') for d in data['trade_date']]

ma5 = MA(data['close'], timeperiod=5)
ma5 = [round(v, 3) for v in ma5]

wma5 = WMA(data['close'], timeperiod=5)
wma5 = [round(v, 3) for v in wma5]

line = Line()
line.add_xaxis(trade_date)
line.add_yaxis('ma5', ma5, label_opts=opts.LabelOpts(is_show=False), is_smooth=True, symbol='none')
line.add_yaxis('wma5', wma5, label_opts=opts.LabelOpts(is_show=False), is_smooth=True, symbol='none')
line.add_yaxis('close', data['close'], label_opts=opts.LabelOpts(is_show=False), is_smooth=True, symbol='none')

line.set_global_opts(xaxis_opts=opts.AxisOpts(is_scale=True),
                     yaxis_opts=opts.AxisOpts(
                         is_scale=True,
                         splitarea_opts=opts.SplitAreaOpts(
                             is_show=True, areastyle_opts=opts.AreaStyleOpts(opacity=1)
                         ),
                     ),
                      datazoom_opts=[opts.DataZoomOpts(pos_bottom="-2%", filter_mode='none')],
                      title_opts=opts.TitleOpts(title="Kline-MarkLine"))

x_index, _, y_extrema = extrema(data, cmp_func=np.greater)

scatter = Scatter()
scatter.add_xaxis(xaxis_data=x_index)
scatter.add_yaxis(
        series_name="极大值",
        y_axis=y_extrema,
        label_opts=opts.LabelOpts(is_show=False), symbol_size=8
    )
line.overlap(scatter)

x_index, _, y_extrema = extrema(data, cmp_func=np.less)
# scatter = Scatter()
# scatter.add_xaxis(xaxis_data=x_index)
# scatter.add_yaxis(
#         series_name="极小值",
#         y_axis=y_extrema,
#         label_opts=opts.LabelOpts(is_show=False),
#     )
scatter = plot_chart(Scatter, x_index, y_extrema, '极小值')
line.overlap(scatter)


a, b, score, x_index, y_index = linear_fitting(data)

line2 = Line()
line2.add_xaxis(list(x_index))
line2.add_yaxis('线性拟合', y_index, label_opts=opts.LabelOpts(is_show=False), is_smooth=True)
line.overlap(line2)

line.render_notebook()


In [5]:
data = data_fund
x = np.array(data['close'])
xmax = signal.argrelextrema(x, np.greater)
ymax = x[xmax[0]]
xmin = signal.argrelextrema(x, np.less)
ymin = x[xmin[0]]

#xmax = np.array(trade_date)[xmax[0]]
#xmin = np.array(trade_date)[xmin[0]]


#转换成numpy的ndarray数据格式，n行1列,LinearRegression需要列格式数据，如下：
X_train = np.array(xmax[0]).reshape((len(xmax[0]), 1))
Y_train = np.array(ymax).reshape((len(ymax), 1))
#新建一个线性回归模型，并把数据放进去对模型进行训练
lineModel = LinearRegression()
lineModel.fit(X_train, Y_train)

#用训练后的模型，进行预测
Y_predict = lineModel.predict(X_train)

#coef_是系数，intercept_是截距，拟合的直线是y=ax+b
a = lineModel.coef_[0][0]
b = lineModel.intercept_[0]
print("y=%.4f*x+%.4f" % (a,b))

#对回归模型进行评分，这里简单使用训练集进行评分，实际很多时候用其他的测试集进行评分
print("得分", lineModel.score(X_train, Y_train))

Y_predict

tt = [x for v in Y_predict for x in v]



y=0.0011*x+1.0983
得分 0.30868520562280444


In [6]:
a, b, score, x_index, y_index0 = linear_fitting(data)
print('a={}, b={}, score={}'.format(a, b, score))



a=0.0013302173747732295, b=1.0762680521004204, score=0.31522599486705605


In [ ]:
codes = await db_fund.load_fund_info(filter={'type': {'$regex': '场内'}}, projection=['code', 'short_name'])
# codes = await db_stock.load_stock_info(projection=['code', 'name'])

select = []
for item in codes.to_dict('records'):
    kdata = await db_fund.load_fund_daily(filter={'code': item['code']}, limit=30, sort=[('trade_date', -1)])
#     kdata = await db_stock.load_stock_daily(filter={'code': item['code']}, limit=30, sort=[('trade_date', -1)])
    if kdata is None or kdata.shape[0] < 30:
        continue
    kdata = kdata[::-1]
        
    a, b, score, x_index, y_index = linear_fitting(kdata)
    if a > 1 and score > 0.7:
        print('got you: a={}, score={}, {}({})'.format(a, score, item['name'], item['code']))
        select.append(item)
        
print('done')

In [2]:
# data = await db_fund.load_fund_daily(filter={'code': '510150'}, limit=30, sort=[('trade_date', -1)])
data = await db_stock.load_stock_daily(filter={'code': 'sz300999'}, limit=20, sort=[('trade_date', -1)])
data = data[::-1]
kline = my_plot(data)
kline.render_notebook()


In [7]:
data_fund = await db_fund.load_fund_daily(filter={'code': '512690'}, limit=60, sort=[('trade_date', -1)])
data_fund = data_fund[::-1]

name_fund = await db_fund.load_fund_info(filter={'code': '512690'}, projection=['short_name'])
data_fund['name'] = name_fund.iloc[0]['short_name']

print('done')

done


In [2]:
inst = strategies['test.TopCode'](db=db_fund)
await inst.init(sort_by='score')

True

In [3]:
select = await inst.select()

done


In [10]:
select

In [4]:

df = pd.DataFrame([x for x in filter(lambda v: v['coef'] > 0.01 and v['score'] > 0.7 and v['rise'] > 1, select)])
df

,coef,score,rise,code,name
0,0.0332,0.9412,29.19,510630,华夏消费ETF
1,0.0212,0.9350,33.67,512690,鹏华中证酒ETF
2,0.0504,0.9345,24.20,510150,招商上证消费80ETF
3,0.0118,0.9319,22.07,512170,华宝中证医疗ETF
4,0.0223,0.8957,27.13,512600,嘉实中证主要消费ETF
5,0.0151,0.8928,16.20,510660,华夏医药ETF
6,0.0203,0.8904,34.09,159806,国泰中证新能源汽车ETF
7,0.0161,0.8888,34.22,515030,华夏中证新能源汽车ETF
8,0.0198,0.8756,25.19,159928,汇添富中证主要消费ETF
9,0.0120,0.8727,28.86,512770,华夏战略新兴成指ETF


In [15]:
data_fund = await db_fund.load_fund_daily(filter={'code': '515790'}, limit=30, sort=[('trade_date', -1)])
data_fund = data_fund[::-1]

a, b, score, x_index, y_index0 = linear_fitting(data_fund)
print('a={}, b={}, score={}'.format(a, b, score))
# my_plot(data_fund).render_notebook()

data_fund

a=0.007950310559006217, b=1.0728571428571427, score=0.2665373890918801


,volume,open,high,low,close,turnover,trade_date,code
12,752675700,1.016,1.042,1.013,1.031,0.0,2020-12-18,515790
11,1192576200,1.062,1.086,1.054,1.083,0.0,2020-12-21,515790
10,1274099600,1.083,1.092,1.054,1.056,0.0,2020-12-22,515790
9,1524729100,1.072,1.133,1.071,1.119,0.0,2020-12-23,515790
8,1272699100,1.125,1.129,1.101,1.112,0.0,2020-12-24,515790
7,1296921800,1.109,1.163,1.105,1.157,0.0,2020-12-25,515790
6,1662164700,1.167,1.169,1.138,1.160,0.0,2020-12-28,515790
5,3099243500,1.151,1.152,1.086,1.090,0.0,2020-12-29,515790
4,1356548000,1.099,1.123,1.097,1.113,0.0,2020-12-30,515790
3,1150560500,1.124,1.147,1.116,1.136,0.0,2020-12-31,515790


In [8]:
net_df = await db_fund.load_fund_net(filter={'dividend': {'$regex': '每份派现金'}}, sort=[('code', 1), ('trade_date', -1)])
group_df = net_df.groupby('code')
net_df.fillna(method='bfill', inplace=True)

In [67]:

net_df['year'] = net_df['trade_date'].apply(lambda d: d.strftime('%Y'))
def apply_func(row):
    s = row['dividend'].replace('每份派现金', '')
    s = s.replace('元', '')
    m = float(s)
    
    if row['net'] > 0:
        row['rate'] = round(m / row['net'] * 100, 2)
    
    return row
    
net_df = net_df.apply(func=apply_func, axis=1)


In [70]:
net_df

,apply_status,code,day_grow_rate,dividend,net,net_accumulate,rate,redeem_status,trade_date,year,cycle
0,开放申购,000001,0.51,每份派现金0.0500元,1.3230,3.8340,3.78,开放赎回,2020-12-17,2020,59
22,限制大额申购,000006,-1.31,每份派现金0.0880元,1.8099,1.8979,4.86,开放赎回,2020-10-19,2020,-66
24,开放申购,000011,0.05,每份派现金0.8910元,18.9760,25.7800,4.70,开放赎回,2020-12-24,2020,28
25,开放申购,000011,-0.63,每份派现金0.9730元,19.4580,25.3710,5.00,开放赎回,2020-11-26,2020,-27
42,限制大额申购,000017,0.93,每份派现金0.4980元,2.1090,3.7010,23.61,开放赎回,2020-12-23,2020,258
...,...,...,...,...,...,...,...,...,...,...,...
6409,限制大额申购,690005,0.50,每份派现金0.1450元,2.8740,3.4010,5.05,开放赎回,2020-11-10,2020,-44
6440,限制大额申购,762001,0.18,每份派现金0.4347元,1.7865,3.3132,24.33,开放赎回,2020-12-24,2020,10
6447,限制大额申购,770001,1.23,每份派现金0.0680元,1.4426,2.3826,4.71,开放赎回,2020-12-14,2020,77
6448,限制大额申购,770001,0.21,每份派现金0.0720元,1.4159,2.2879,5.09,开放赎回,2020-09-28,2020,80


In [69]:
net_df = net_df[net_df['year'] == '2020']
net_df['cycle'] = net_df['trade_date'].diff(-1).fillna(timedelta(days=0)).apply(lambda v: v.days)
group_df = net_df[net_df['year'] == '2020'].groupby('code')


In [42]:
r = group_df.agg({'rate': 'sum', 'year': 'count'})

In [44]:
r.sort_values(by='rate', ascending=False)

,rate,year
code,,
257030,69.54,2
310308,63.34,4
217005,59.62,2
610002,59.44,3
160726,58.99,1
...,...,...
159816,0.04,1
168601,0.00,1
005746,0.00,1


In [2]:
inst = strategies['test.HighDiv'](db=db_fund)
await inst.init(year='2020')

True

In [3]:
div_df = await inst.select()

done


In [5]:
div_df[div_df['rate'] > 30]


,rate,count
code,,
257030,69.54,2
310308,63.34,4
217005,59.62,2
610002,59.44,3
009317,50.01,2
007012,49.70,2
006109,49.27,2
007013,49.02,2
519694,48.91,4


In [6]:
net_df = await db_fund.load_fund_net(filter={'dividend': {'$regex': '每份派现金'}, 'code': '257030'}, sort=[('code', 1), ('trade_date', -1)])
net_df

,code,trade_date,net,net_accumulate,day_grow_rate,apply_status,redeem_status,dividend
0,257030,2020-12-28,1.062,2.696,0.06,限制大额申购,开放赎回,每份派现金0.4900元
1,257030,2020-03-16,0.936,2.080,-4.23,限制大额申购,开放赎回,每份派现金0.2190元
2,257030,2019-09-19,1.145,2.070,0.33,开放申购,开放赎回,每份派现金0.0580元
3,257030,2018-06-27,1.064,1.931,-2.53,开放申购,开放赎回,每份派现金0.0550元
4,257030,2017-12-18,1.199,2.011,-0.14,限制大额申购,开放赎回,每份派现金0.2360元
5,257030,2016-03-08,1.070,1.646,0.09,开放申购,开放赎回,每份派现金0.0200元
6,257030,2015-01-22,1.109,1.665,0.51,开放申购,开放赎回,每份派现金0.2600元
7,257030,2014-01-14,0.980,1.276,1.53,开放申购,开放赎回,每份派现金0.0150元
8,257030,2011-03-22,1.005,1.286,0.49,开放申购,开放赎回,每份派现金0.0210元
9,257030,2010-12-14,1.076,1.336,0.68,开放申购,开放赎回,每份派现金0.1000元


In [15]:
for k, v in group_df.groups.items():
    print(net_df.iloc[v])
    break

      code trade_date    net  net_accumulate  day_grow_rate apply_status  \
0   000001 2020-12-17  1.323           3.834           0.51         开放申购   
1   000001 2019-11-06  1.065           3.526          -0.80         开放申购   
2   000001 2017-12-07  1.100           3.511          -1.01         开放申购   
3   000001 2016-12-26  1.041           3.372           0.47         开放申购   
4   000001 2015-11-20  1.396           3.697           1.56         开放申购   
5   000001 2014-11-20  1.133           3.334          -0.16       限制大额申购   
6   000001 2010-09-07  1.113           3.214           0.35       限制大额申购   
7   000001 2009-12-25  1.445           3.226           0.13         开放申购   
8   000001 2008-01-23  1.686           3.367           4.24         开放申购   
9   000001 2007-01-12  0.979           2.160          -1.31         开放申购   
10  000001 2006-09-27  1.297           1.647           0.84         开放申购   
11  000001 2006-08-31  1.265           1.585           0.31         开放申购   
12  000001 2